<a href="https://colab.research.google.com/github/peppopi/Piratas/blob/main/Lora_Chekpoint_Merger_BasicV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Aca se Pone el Link del Modelo Base Usar, se recomienda ir poniendo varios, se sovrescribe automaticamente
import os
import requests
from tqdm import tqdm

ruta_carpeta = "/content/models/"

def descargar_archivo(url, ruta_carpeta, nombre_archivo):
    if not os.path.exists(ruta_carpeta):
        os.makedirs(ruta_carpeta)

    ruta_completa_archivo = os.path.join(ruta_carpeta, nombre_archivo)

    response = requests.get(url, stream=True)
    total_size = int(response.headers.get("content-length", 0))

    with open(ruta_completa_archivo, "wb") as archivo:
        progress_bar = tqdm(total=total_size, unit="B", unit_scale=True)
        for data in response.iter_content(chunk_size=4096):
            archivo.write(data)
            progress_bar.update(len(data))
        progress_bar.close()

# URL del archivo que deseas descargar
url = "https://civitai.com/api/download/models/15236" #@param {type:"string"}
# Nombre del archivo en tu entorno de ejecución
nombre_archivo = "Nomana.safetensors"

# Descargar el archivo
descargar_archivo(url, ruta_carpeta, nombre_archivo)



In [ ]:
#Se descargan dependencias:
!git clone https://github.com/kohya-ss/sd-scripts.git
!cd /content/sd-scripts && pip install -r requirements.txt


#Se descargan los Loras:
!wget https://civitai.com/api/download/models/47143 -O ArchivoUNO.safetensors
!wget https://civitai.com/api/download/models/90716 -O ArchivoDOS.safetensors
!wget https://civitai.com/api/download/models/52086 -O ArchivoTRES.safetensors
!wget https://civitai.com/api/download/models/55777 -O ArchivoCUATRO.safetensors
#Formato
# !wget [LINK] -O [Nombre De Archivo].safetensors (NO hace falta poner el verdaero, es inventado)

In [ ]:
# Crea el Chekpoint, es decir hace el mergin y combina los Loras con un modelo
!python sd-scripts/networks/merge_lora.py \
--sd_model /content/models/Nomana.safetensors \
--save_to /content/Gato.safetensors \
--models \
    /content/ArchivoUNO.safetensors \
    /content/ArchivoDOS.safetensors \
    /content/ArchivoTRES.safetensors \
    /content/ArchivoCUATRO.safetensors \
--ratios 1 1 1 1

#Simpre sera Nomana.safetensors el Modelo, pero recorda que si o si el modelo debe tener VAE para usarlo aca, sino da error.
#Donde se guardara
#Donde se guardara
#Aca van los nombres de los LORAS
#Aca pones que tanto se pondera cada Lora, un 1 es ponderar todo, un 0.5 es ponderar la mitad, un 0.25 es ponderar un cuarto del LORA

#Para agregar un Lora hay que seguir la siguiente regla
#Se agrega un NUMERO en el --ratios POR LORA ==> 5 LORAS, 5 NUMEROS RATIOS ==> --ratios 1 1 1 1 1
#Se agrega una carpeta de descarga por  /content/[Nombre De Archivo].safetensors "\" <--Este indica que el codigo continua en la linia de abajo, si ejas un espacio vacio y no pones esto da error



In [ ]:
#Fusion De Loras
!python sd-scripts/networks/merge_lora.py \
--save_precision fp16 \
--save_to /content/Gato.safetensors \
--models \
    /content/ArchivoUNO.safetensors \
    /content/ArchivoDOS.safetensors \
    /content/ArchivoTRES.safetensors \
    /content/ArchivoCUATRO.safetensors \
--ratios 0.25 0.25 0.25 0.2


---
# **Useful Utilities (Optional)**

*Stuff that might be useful*

## Upload stuff
*Quickly upload stuff like merged models to huggingface*

Run cell `1,2,3`

You only need to choose one between `3.1,3.2,and 3.3`

<small>Forked from LYNN


In [ ]:
#@title 1. Login to Huggingface hub (LEE LAS INTRUCIONES)
try:
  hub_ok # My packaged dep didn't contains this (but stil have the folder)
except:
  print("Setting up huggingface_hub...")
  !pip install --force-reinstall -qqqq huggingface_hub
  hub_ok = True
from IPython.display import clear_output
from huggingface_hub import login
clear_output()

#@markdown 1. Of course, you need a Huggingface account first.
#@markdown 2. To create a huggingface token, go to [this link](https://huggingface.co/settings/tokens), then `create new token` or copy available token with the `Write` role.

write_token = "" #@param {type:"string"}
login(write_token, add_to_git_credential=True)

In [ ]:
#@title 2. Setup Repo (Ya biene todo armado solo dale clik)
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from huggingface_hub import HfApi


api = HfApi()
user = api.whoami(write_token)

#@markdown #### If your model repo didn't exist, it will automatically create your repo.
repo_name = "Nombre" #@param{type:"string"}
make_this_repo_private_if_not_exist = False #@param{type:"boolean"}
clone_with_git = True #@param{type:"boolean"}

model_repo = user['name']+"/"+repo_name.strip()

validate_repo_id(model_repo)

if repo_name != "":
  try:
      api.create_repo(repo_id=model_repo,
                      private=make_this_repo_private_if_not_exist)
      print("Model Repo didn't exists, creating repo")
      print("Model Repo: ",model_repo,"created!\n")

  except HfHubHTTPError as e:
      print(f"Model Repo: {model_repo} exists, skipping create repo\n")

if clone_with_git:
  !git lfs install --skip-smudge
  !export GIT_LFS_SKIP_SMUDGE=1
  !git clone https://huggingface.co/{model_repo} /content/{repo_name}

In [ ]:
#@title 3.2 Upload via huggingface_hub (Manual way) (Ya biene todo armado solo dale clik)

#@markdown All paths for easier access:<br>
#@markdown /content/sdw/models/Stable-diffusion<br>
#@markdown /content/sdw/models/VAE<br>
#@markdown /content/sdw/models/Lora<br>
#@markdown /content/sdw/embeddings<br>
#@markdown /content/sdw/models/hypernetwork<br>
from huggingface_hub import HfApi
from pathlib import Path

api = HfApi()
file_path = "/content/Gato.safetensors" #@param {type :"string"}
commit_message = "Upload with \uD83D\uDE80\uD83E\uDD17 NoCrypt's --> El que se armo este codigo tan simple" #@param {type :"string"}

if file_path != "":
  path_obj = Path(file_path)
  trained_model = path_obj.parts[-1]

  print(f"Uploading {trained_model} to https://huggingface.co/"+model_repo)
  print(f"Please wait...")

  api.upload_file(
      path_or_fileobj=file_path,
      path_in_repo=trained_model,
      repo_id=model_repo,
      commit_message=commit_message,
  )

  print(f"Upload success, located at https://huggingface.co/"+model_repo+"/blob/main/"+trained_model+"\n")